# The London Railway Network

The cell below defines the abstract class whose API you will need to impement. Do NOT modify it.

In [1]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    # returns the minimum number of stops to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # numStops : int
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    # returns the minimum distance in miles to connect station "fromS" to station  "toS"
    # fromS : str
    # toS : str
    # minDistance : float
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    # given an unordered list of station names, returns a new railway line 
    # (represented as a list of adjacent station names), connecting all such stations 
    # and such that the sum of the distances (in miles) between adjacent stations is minimised
    # inputList : set<str>
    # outputList : list<str>
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

Use the cell below to define any data structure and auxiliary python function you may need. Leave the implementation of the main API to the next code cell instead.

In [2]:
import math
import csv

class MinHeap:
    # A MinHeap has Time Complexity ~ O(Log N) for both
    # insertion and removal
    # Time Complexity ~ O(1) for extracting minimum
    def __init__(self):
        # A base key and value on the heap to help with implementation
        self.keys = ["NULL"]
        self.heap = [0]

    # Append value to end of the heap and float it up to the proper position
    def push(self, key, value):
        try:
            self.keys.index(key)
        except ValueError:
            self.keys.append(key)
            self.heap.append(value)
            self.float_up(len(self.heap) - 1)
            return True
        self.pop_by_key(key)
        return self.push(key, value)

    # Checks to see if there is at least one value on the heap and if so
    # returns that value or else False to confirm empty heap
    def peek(self):
        if self.keys[1] and self.heap[1]:
            return [self.keys[1], self.keys.heap[1]]
        return False

    def pop(self):
        # If there are two or more value on the heap, then swap minimum value
        # to the end of the heap before popping it off. Then float down value
        # slotted into the top position
        if len(self.heap) > 2:
            self.swap(1, len(self.heap) - 1)
            minimum_key = self.keys.pop()
            minimum_value = self.heap.pop()
            self.min_heapify(1)
        # If there is only one value on the heap then the top value can just
        # simply be popped off
        elif len(self.heap) == 2:
            minimum_key = self.keys.pop()
            minimum_value = self.heap.pop()
        # If the heap is empty then return False
        else:
            minimum_key = False
            minimum_value = False
        return [minimum_key, minimum_value]

    def pop_by_key(self, key):
        try:
            index = self.keys.index(key)
        except ValueError:
            return [False, False]

        if len(self.heap) > 2:
            self.swap(index, len(self.heap) - 1)
            minimum_key = self.keys.pop(len(self.heap) - 1)
            minimum_value = self.heap.pop(len(self.heap) - 1)
            self.min_heapify(index)
        elif len(self.heap) == 2:
            minimum_key = self.keys.pop(index)
            minimum_value = self.heap.pop(index)

        return [minimum_key, minimum_value]

    def swap(self, i, j):
        self.keys[i], self.keys[j] = self.keys[j], self.keys[i]
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    # Float up a value at the end of a heap to its proper position
    def float_up(self, index):
        parent = index // 2
        # No need for floating to be done, already in right position
        if index <= 1:
            return
        elif self.heap[index] < self.heap[parent]:
            self.swap(index, parent)
            self.float_up(parent)

    # Bubbles down value inserted at the top of the heap to its
    # correct position
    def min_heapify(self, index):
        left = index * 2
        right = index * 2 + 1
        lowest = index
        if len(self.heap) > left and self.heap[lowest] > self.heap[left]:
            lowest = left
        if len(self.heap) > right and self.heap[lowest] > self.heap[right]:
            lowest = right
        if lowest != index:
            self.swap(index, lowest)
            self.min_heapify(lowest)

            
class Deque:
    # Time Complexity ~ O(1) for all enqueue, peek and pop operations
    # This is a double ended queue implementation but can also be used
    # as a single ended queue with same time complexities for all operations
    def __init__(self):
        self.items = []

    def is_empty(self):
        return self.items == []

    def length(self):
        return len(self.items)

    def add_first(self, item):
        self.items.append(item)

    def add_last(self, item):
        self.items.insert(0, item)

    def peek_first(self):
        if self.items:
            return self.items[-1]

    def remove_first(self):
        if self.items:
            self.items.pop(-1)

    def peek_last(self):
        if self.items:
            return self.items[0]

    def remove_last(self):
        if self.items:
            self.items.pop(0)

    def peek(self, index):
        if self.items:
            return self.items[index]

    def remove(self, index):
        if self.items:
            self.items.pop(index)


# A class that defines a node in a stations graph
class Station:
    def __init__(self, station_name, latitude, longitude):
        self.name = station_name
        self.latitude = float(latitude)
        self.longitude = float(longitude)
        self.connections = []

    def add_neighbor(self, station):
        # Check to see connection does not already exist
        if station in self.connections:
            return
        self.connections.append(station)


# This graph is and adjacency list implementation of an undirected
# and weighted graph
class Graph:
    def __init__(self):
        self.stations = {}

    # Each station is only added to the graph at most once
    def add_station(self, station):
        if isinstance(station, Station) and station.name not in self.stations:
            self.stations[station.name] = station
            return True
        return False

    def add_connection(self, station1, station2):
        if station1 in self.stations and station2 in self.stations:
            self.stations[station1].add_neighbor(station2)
            self.stations[station2].add_neighbor(station1)
            # The distance between two stations are not calculated now
            # but only when required as this prevents unnecessary computations
            return True
        return False

    def print_graph(self):
        for key in sorted(list(self.stations.keys())):
            print(key + str(self.stations[key].connections))


# This is a modified Union Find algorithm to suit new railway lines needs
# The union by rank and path compression optimisations have been applied here
# giving both the union and find operations a time complexity of O(log N)
class UnionFind:
    def __init__(self):
        self.distance_to_parent_stations = {}
        self.parent_stations = {}

    # The union by rank optimisation has been applied here
    # The smaller depth tree is always attached under the root of the deeper tree
    def union(self, *stations):  # *stations allows to behave as a tuple
        root_stations = [self.find(station) for station in stations]
        deepest_station = max([(self.distance_to_parent_stations[root], root) for root in root_stations])[1]
        for root in root_stations:
            if root != deepest_station:
                self.distance_to_parent_stations[deepest_station] += self.distance_to_parent_stations[root]
                self.parent_stations[root] = deepest_station

    # The path compression optimisation has been applied here
    def find(self, item):
        if item not in self.parent_stations:
            self.parent_stations[item] = item
            self.distance_to_parent_stations[item] = 1
            return item

        # This finds the path of parent stations that leads to the root station
        path_to_root = [item]
        root = self.parent_stations[item]
        while path_to_root[-1] != root:
            path_to_root.append(root)
            root = self.parent_stations[root]

        # The path is compressed so that intermediate nodes are not traversed again
        # This flattens the tree when find is called again
        for parent in path_to_root:
            self.parent_stations[parent] = root
        return root

    
def haversine(longitude1, latitude1, longitude2, latitude2) -> float:
    """
    The haversine formula determines the great-circle distance between
    two points on a sphere given their longitudes and latitudes.
    Referenced from - https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    # Converts from degrees to radians
    longitude1, latitude1, longitude2, latitude2 = map(math.radians, [longitude1, latitude1, longitude2, latitude2])
    # Haversine formula
    difference_longitude = longitude2 - longitude1
    difference_latitude = latitude2 - latitude1
    a = math.sin(difference_latitude / 2) ** 2 + math.cos(latitude1) * math.cos(latitude2) * math.sin(difference_longitude / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    r = 3959.87433  # Radius of earth in miles
    return c * r


def compute_distance_between_stations(station1, station2) -> float:
    return haversine(station1.longitude, station1.latitude, station2.longitude, station2.latitude)

Use the cell below to implement the requested API.

In [3]:
import csv


class LondonRailwayMapper(AbstractLondonRailwayMapper):
    
    def __init__(self):
        self.station_filename = "londonstations.csv"
        self.connections_filename = "londonrailwaylines.csv"
        self.stations_map = None     
        
    def loadStationsAndLines(self):   
        graph = Graph()

        file_contents = csv.reader(open(self.station_filename, "r"), delimiter=',')

        number_of_lines = 0
        for row in file_contents:
            if number_of_lines > 0:
                graph.add_station(Station(row[0], row[1], row[2]))
            number_of_lines += 1

        file_contents = csv.reader(open(self.connections_filename, "r"), delimiter=',')

        number_of_lines = 0
        for row in file_contents:
            if number_of_lines > 0:
                graph.add_connection(row[1], row[2])
            number_of_lines += 1

        self.stations_map = graph

    def minStops(self, fromS, toS):     
        numStops = -1
        station_from = fromS
        station_to = toS
        
        # BFS from start vertex using graph represented by adjacency list
        # Time Complexity ~ O(V + E)
        # A dictionary is used to store previous stations, distance traversed
        # so far and whether the station has been visited as it has O(1) lookup
        # and insertion time complexity

        previous_stations = {}
        number_of_stops = {}
        visited = {}

        for key in self.stations_map.stations:
            previous_stations[key] = -1
            number_of_stops[key] = 0
            visited[key] = False

        queue_for_bfs = Deque()
        # A double ended queue implementation is being used however this
        # queue is used as a single ended queue. A double ended queue was implemented
        # for generality.
        visited[station_from] = True
        queue_for_bfs.add_first(station_from)
        
        global test_stations_stops # DO NOT MODIFY THIS - USED TO TEST MinStops BELOW
        test_stations_stops = 0
        while not queue_for_bfs.is_empty():
            current_station = queue_for_bfs.peek_last()
            queue_for_bfs.remove_last()
            for connected_station in self.stations_map.stations[current_station].connections:
                if visited[connected_station]:
                    continue
                # Update the visited, number_of_stops and previous_stations dictionaries
                # the connected station
                visited[connected_station] = True
                test_stations_stops += 1
                number_of_stops[connected_station] = number_of_stops[current_station] + 1
                previous_stations[connected_station] = current_station
                queue_for_bfs.add_first(connected_station)

                if connected_station == station_to:
                    numStops = number_of_stops[station_to]
                    return numStops

        return numStops
    
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        station_from = fromS
        station_to = toS
        
        distance_traversed = {}
        visited = {}
        priority_queue = MinHeap()

        for key in self.stations_map.stations:
            distance_traversed[key] = float("inf")
            visited[key] = False

        distance_traversed[station_from] = 0
        current_station = station_from
        priority_queue.push(current_station, 0)

        # Dijkstra's algorithm is used here and an adjacency list
        # representation is being used here as the time complexity is
        # O(E log V). A Priority Queue implemented as a Min Heap is used
        # as extracting minimum operations is O(Log V) for a Min Heap.
        global test_stations_distance # DO NOT MODIFY THIS - USED TO TEST MinDistance BELOW
        test_stations_distance = 0
        while current_station != station_to:
            for connection in self.stations_map.stations[current_station].connections:
                if not visited[connection]:
                    total_distance = distance_traversed[current_station] + compute_distance_between_stations(
                        self.stations_map.stations[current_station], self.stations_map.stations[connection])
                    if total_distance < distance_traversed[connection]:
                        distance_traversed[connection] = total_distance
                        priority_queue.push(connection, total_distance)

            visited[current_station] = True
            test_stations_distance += 1
            priority_queue.pop_by_key(current_station)
            current_station = priority_queue.pop()[0]
        minDistance = distance_traversed[station_to]

        return minDistance
    
    def newRailwayLine(self, inputList):
        outputList = []

        def clone_graph(names, base_graph):
            new_graph = {}  # Adjacency list graph but all distances are initially calculated
            # Time complexity to clone ~ O(N^2)
            for key1 in names:
                for key2 in names:
                    if key1 != key2:
                        if key1 not in new_graph:
                            new_graph[key1] = {}
                        new_graph[key1][key2] = compute_distance_between_stations(base_graph.stations[key1], base_graph.stations[key2])

            return new_graph

        def construct_mst(base_graph):
            # Kruskal's algorithm is being used here and the union find
            # algorithm to check for cycles when adding each edge
            # Time complexity ~ O(E Log V) as the complete graph with all stations included is being used
            minimum_spanning_tree = []
            sub_trees = UnionFind()
            sorted_base_graph = sorted((base_graph[key1][key2], key1, key2) for key1 in base_graph for key2 in base_graph[key1])
            # Time complexity of python sorted ~ O(N Log N) in average and worst case
            # This is guaranteed because accessing elements in a dictionary has cost O(1)
            for weight, stationA, stationB in sorted_base_graph:
                # Check to see if both stations are not in the same tree, i.e. they don't have the same root
                # This is to make sure the edge does not form any cycles
                if sub_trees.find(stationA) != sub_trees.find(stationB):
                    minimum_spanning_tree.append((stationA, stationB, weight))
                    sub_trees.union(stationA, stationB)

            return minimum_spanning_tree

        def find_odd_vertexes(tree):
            # Iterate through the mst to find which stations have an odd number of connections to other stations
            # Dictionary is used to store the total number of connections per station for O(1) lookup and O(1) insertion
            # Time complexity ~ O(N)
            number_connections_for_stations = {}
            stations_odd_connections = []
            for connection in tree:
                if connection[0] not in number_connections_for_stations:
                    number_connections_for_stations[connection[0]] = 0

                if connection[1] not in number_connections_for_stations:
                    number_connections_for_stations[connection[1]] = 0

                number_connections_for_stations[connection[0]] += 1
                number_connections_for_stations[connection[1]] += 1

            # Check if the stations have an odd or even number of connections
            for station in number_connections_for_stations:
                if number_connections_for_stations[station] % 2 != 0:
                    stations_odd_connections.append(station)

            return stations_odd_connections

        def minimum_weight_matching(tree, base_graph, odd_stations):
            while odd_stations:
                minimum_distance_between_pair = float("inf")
                minimum_matching_pair = []
                # Iterate through all odd stations and pairs up ones with
                # lowest distance. Time complexity ~ O(N^3)
                for stationA in odd_stations:
                    for stationB in odd_stations:
                        if stationA != stationB and base_graph[stationA][stationB] < minimum_distance_between_pair:
                            minimum_distance_between_pair = base_graph[stationA][stationB]
                            minimum_matching_pair = (stationA, stationB, minimum_distance_between_pair)

                # Once a pair has been matched, remove those stations from
                # odd stations
                tree.append(minimum_matching_pair)
                odd_stations.remove(minimum_matching_pair[0])
                odd_stations.remove(minimum_matching_pair[1])
            return tree

        def find_start_station(names, base_graph):
            # This method iterates through all the stations in the input set and
            # computes an average latitude and average longitude
            average_latitude = 0
            average_longitude = 0

            for key1 in names:
                average_latitude += base_graph.stations[key1].latitude
                average_longitude += base_graph.stations[key1].longitude

            average_latitude = average_latitude / len(names)
            average_longitude = average_longitude / len(names)

            maximum_displacement_from_center = float("-inf")
            start_station = None

            # All stations are iterated again to find the station that is furthest from the center
            # This is set as the start station to minimise the total track length
            for key1 in names:
                displacement_from_center = haversine(average_longitude, average_latitude, base_graph.stations[key1].longitude, base_graph.stations[key1].latitude)
                if displacement_from_center > maximum_displacement_from_center:
                    maximum_displacement_from_center = displacement_from_center
                    start_station = key1

            return start_station

        def find_neighbours(matched_mst):
            # The edges to and from each vertex
            # is computed here. A dictionary is being used here
            # as lookup and insertion has Time complexity ~ O(1)
            close_stations = {}
            for connection in matched_mst:
                if connection[0] not in close_stations:
                    close_stations[connection[0]] = []

                if connection[1] not in close_stations:
                    close_stations[connection[1]] = []

                close_stations[connection[0]].append(connection[1])
                close_stations[connection[1]].append(connection[0])

            return close_stations

        def construct_eulerian_path(matched_mst, close_stations, start_station):
            euler_path = [close_stations[start_station][0]]

            while len(matched_mst) > 0:
                for count, station in enumerate(euler_path):
                    if len(close_stations[station]) > 0:
                        break

                while len(close_stations[station]) > 0:
                    neighbour = close_stations[station][0]

                    remove_edge(matched_mst, station, neighbour)

                    del close_stations[station][(close_stations[station].index(neighbour))]
                    del close_stations[neighbour][(close_stations[neighbour].index(station))]

                    count += 1
                    euler_path.insert(count, neighbour)

                    station = neighbour

            return euler_path

        def remove_edge(matched_mst, station1, station2):
            for count, value in enumerate(matched_mst):
                if (value[0] == station2 and value[1] == station1) or (value[0] == station1 and value[1] == station2):
                    del matched_mst[count]

            return matched_mst

        def take_shortcuts(edges, start_station):
            # This method skips past repeating stations
            # Time complexity ~ O(N)
            final = []
            repeated = {}
            for edge in edges:
                if edge not in repeated:
                    final.append(edge)
                    repeated[edge] = 1

            # This method reorders the final path to start from
            # the computed start station to minimise travelled distance
            # Time complexity ~ O(N)
            for index in range(len(final)):
                if final[index] == start_station:
                    return final[index:] + final[:index]
            return final
        
        DEBUG = False
        graph = clone_graph(inputList, self.stations_map)
        if DEBUG: print("Graph: \n", graph, end='\n\n')
        mst = construct_mst(graph)
        if DEBUG: print("MST: \n", mst, end='\n\n')
        odd_vertexes = find_odd_vertexes(mst)
        if DEBUG: print("Odd Vertexes: \n", odd_vertexes, end='\n\n')
        mst = minimum_weight_matching(mst, graph, odd_vertexes)
        if DEBUG: print("Min weight matching: \n", mst, end='\n\n')
        start = find_start_station(inputList, self.stations_map)
        if DEBUG: print("Start station: \n", start, end='\n\n')
        neighbours = find_neighbours(mst)
        if DEBUG: print("Neighbours: \n", neighbours, end='\n\n')
        eulerian_tour = construct_eulerian_path(mst, neighbours, start)
        if DEBUG: print("Eulerian tour: \n", eulerian_tour, end='\n\n')
        final_tour = take_shortcuts(eulerian_tour, start)
        if DEBUG: print("No. of final stations: ", len(final_tour), "\nFinal tour -> ", end='\n\n')
 
        total_distance = 0
        for i in range(len(final_tour) - 1):
            if DEBUG: print(final_tour[i], "->", final_tour[i + 1], "~", compute_distance_between_stations(self.stations_map.stations[final_tour[i]], self.stations_map.stations[final_tour[i + 1]]))
            total_distance += compute_distance_between_stations(self.stations_map.stations[final_tour[i]], self.stations_map.stations[final_tour[i + 1]])
        if DEBUG: print("Total distance in miles:", total_distance)
        
        outputList = final_tour
        global test_length # DO NOT MODIFY - USED TO TEST NewRailwayLine BELOW
        test_length = total_distance
        return outputList

Use the cell below for all python code needed to test the `LondonRailwayMapper` class above.

In [4]:
import timeit
import random

plotMapper = LondonRailwayMapper()
plotMapper.loadStationsAndLines()

def tablify(string):
    MAX_LENGTH = 30
    while len(string) < MAX_LENGTH:
        string += " "
    return string

def testMinimumDistance(testMapper, citiesToTest, nof_repeats, nof_tests, display):
    if display: print(tablify("No. of stations - traversed"), tablify("Average time taken"), tablify("Minimum distance"))
    for i in range(0, nof_tests):
        from_station = random.choice(citiesToTest)
        to_station = random.choice(citiesToTest)
        start_time = timeit.default_timer()
        min_distance = testMapper.minDistance(from_station, to_station)
        for j in range(1, nof_repeats):
            testMapper.minDistance(from_station, to_station)
        avg_time = (timeit.default_timer() - start_time) / nof_repeats
        if display: print(tablify(str(test_stations_distance)), tablify(str(avg_time)), tablify(str(min_distance)))
    
def testMinimumStops(testMapper, citiesToTest, nof_repeats, nof_tests, display):
    if display: print(tablify("No. of stations - traversed"), tablify("Average time taken"), tablify("Minimum stops"))
    for i in range(0, nof_tests):
        from_station = random.choice(citiesToTest)
        to_station = random.choice(citiesToTest)
        start_time = timeit.default_timer()
        min_stops = testMapper.minStops(from_station, to_station)
        for j in range(1, nof_repeats):
            testMapper.minStops(from_station, to_station)
        avg_time = (timeit.default_timer() - start_time) / nof_repeats
        if display: print(tablify(str(test_stations_stops)), tablify(str(avg_time)), tablify(str(min_stops)))

def testNewRailwayLine(testMapper, citiesToTest, nof_repeats, nof_tests, display):
    if display: print(tablify("No. of stations - input list"), tablify("Average time taken"), tablify("Minimum length"))
    for i in range(0, nof_tests):
        input_list = random.sample(citiesToTest, random.randint(1, len(citiesToTest)))
        start_time = timeit.default_timer()
        for j in range(0, nof_repeats):
            testMapper.newRailwayLine(input_list)
        avg_time = (timeit.default_timer() - start_time) / nof_repeats
        if display: print(tablify(str(len(input_list))), tablify(str(avg_time)), tablify(str(test_length)))
        
# Main code to run tests
NOF_REPEATS = 3
NOF_TESTS = 5
DISPLAY_TEST_TABLE = True # Change to true to see the test results table
citiesToTest = ['Abbey Road', 'Abbey Wood', 'Acton Central', 'Acton Main Line', 'Acton Town', 'Addington Village', 'Addiscombe', 'Albany Park', 'Aldgate', 'Aldgate East', 'Alexandra Palace', 'All Saints', 'Alperton', 'Amersham', 'Ampere Way', 'Anerley', 'Angel', 'Angel Road', 'Archway', 'Arena', 'Arnos Grove', 'Arsenal', 'Avenue Road', 'Baker Street', 'Balham', 'Bank', 'Banstead', 'Barbican', 'Barking', 'Barkingside', 'Barnehurst', 'Barnes', 'Barnes Bridge', 'Barons Court', 'Battersea Park', 'Battersea Power Station', 'Bayswater', 'Beckenham Hill', 'Beckenham Junction', 'Beckenham Road', 'Beckton', 'Beckton Park', 'Becontree', 'Beddington Lane', 'Belgrave Walk', 'Bellingham', 'Belmont', 'Belsize Park', 'Belvedere', 'Bermondsey', 'Berrylands', 'Bethnal Green', 'Bethnal Green Rail', 'Bexley', 'Bexleyheath', 'Bickley', 'Birkbeck', 'Blackfriars', 'Blackheath', 'Blackhorse Lane', 'Blackhorse Road', 'Blackwall', 'Bond Street', 'Borough', 'Boston Manor', 'Bounds Green', 'Bow Church', 'Bow Road', 'Bowes Park', 'Brent Cross', 'Brentford', 'Brentwood', 'Brimsdown', 'Brixton', 'Brockley', 'Bromley North', 'Bromley South', 'Bromley-by-Bow', 'Brondesbury', 'Brondesbury Park', 'Broxbourne', 'Bruce Grove', 'Buckhurst Hill', 'Burnham', 'Burnt Oak', 'Bush Hill Park', 'Bushey', 'Caledonian Road', 'Caledonian Road and Barnsbury', 'Cambridge Heath', 'Camden Road', 'Camden Town', 'Canada Water', 'Canary Wharf', 'Canning Town', 'Cannon Street', 'Canonbury', 'Canons Park', 'Carpenders Park', 'Carshalton', 'Carshalton Beeches', 'Castle Bar Park', 'Caterham', 'Catford', 'Catford Bridge', 'Centrale', 'Chadwell Heath', 'Chafford Hundred', 'Chalfont and Latimer', 'Chalk Farm', 'Chancery Lane', 'Charing Cross', 'Charlton', 'Cheam', 'Chelsfield', 'Chesham', 'Cheshunt', 'Chessington North', 'Chessington South', 'Chigwell', 'Chingford', 'Chipstead', 'Chislehurst', 'Chiswick', 'Chiswick Park', 'Chorleywood', 'Church Street', 'City Thameslink', 'Clapham Common', 'Clapham High Street', 'Clapham Junction', 'Clapham North', 'Clapham South', 'Clapton', 'Clock House', 'Cockfosters', 'Colindale', 'Colliers Wood', 'Coombe Lane', 'Coulsdon South', 'Coulsdon Town', 'Covent Garden', 'Crayford', 'Crews Hill', 'Cricklewood', 'Crofton Park', 'Crossharbour and London Arena', 'Crouch Hill', 'Croxley', 'Crystal Palace', 'Custom House', 'Cutty Sark for Maritime Greenwich', 'Cyprus', 'Dagenham Dock', 'Dagenham East', 'Dagenham Heathway', 'Dalston Junction', 'Dalston Kingsland', 'Debden', 'Denmark Hill', 'Deptford', 'Deptford Bridge', 'Devons Road', 'Dollis Hill', 'Drayton Green', 'Drayton Park', 'Dundonald Road', 'Ealing Broadway', 'Ealing Common', 'Earls Court', 'Earlsfield', 'East Acton', 'East Croydon', 'East Dulwich', 'East Finchley', 'East Ham', 'East India', 'East Putney', 'Eastcote', 'Eden Park', 'Edgware', 'Edgware Road (Bakerloo)', 'Edgware Road (Circle/District/Hammersmith and City)', 'Edmonton Green', 'Elephant and Castle', 'Elm Park', 'Elmers End', 'Elmstead Woods', 'Elstree and Borehamwood', 'Eltham', 'Elverson Road', 'Embankment', 'Emerson Park', 'Enfield Chase', 'Enfield Lock', 'Enfield Town', 'Epping', 'Epsom Downs', 'Erith', 'Essex Road', 'Euston', 'Euston Square', 'Ewell East', 'Ewell West', 'Fairlop', 'Falconwood', 'Farringdon', 'Feltham', 'Fenchurch Street', 'Fieldway', 'Finchley Central', 'Finchley Road', 'Finchley Road and Frognal', 'Finsbury Park', 'Forest Gate', 'Forest Hill', 'Fulham Broadway', 'Fulwell', 'Gallions Reach', 'Gants Hill', 'George Street', 'Gidea Park', 'Gipsy Hill', 'Gloucester Road', 'Golders Green', 'Goldhawk Road', 'Goodge Street', 'Goodmayes', 'Gordon Hill', 'Gospel Oak', 'Grange Hill', 'Grange Park', 'Gravel Hill', 'Great Portland Street', 'Green Park', 'Greenford', 'Greenwich', 'Grove Park', 'Gunnersbury', 'Hackbridge', 'Hackney Central', 'Hackney Downs', 'Hackney Wick', 'Hadley Wood', 'Haggerston', 'Hainault', 'Hammersmith (District)', 'Hammersmith (Met.)', 'Hampstead', 'Hampstead Heath', 'Hampton', 'Hampton Court', 'Hampton Wick', 'Hanger Lane', 'Hanwell', 'Harlesden', 'Harold Wood', 'Harringay', 'Harringay Green Lanes', 'Harrington Road', 'Harrow and Wealdstone', 'Harrow-on-the-Hill', 'Hatch End', 'Hatton Cross', 'Haydons Road', 'Hayes', 'Hayes and Harlington', 'Headstone Lane', 'Heathrow Terminal 4', 'Heathrow Terminal 5', 'Heathrow Terminals 1 2 3', 'Hendon', 'Hendon Central', 'Herne Hill', 'Heron Quays', 'High Barnet', 'High Street Kensington', 'Highams Park', 'Highbury and Islington', 'Highgate', 'Hillingdon', 'Hither Green', 'Holborn', 'Holland Park', 'Holloway Road', 'Homerton', 'Honor Oak Park', 'Hornchurch', 'Hornsey', 'Hounslow', 'Hounslow Central', 'Hounslow East', 'Hounslow West', 'Hoxton', 'Hyde Park Corner', 'Ickenham', 'Ilford', 'Imperial Wharf', 'Island Gardens', 'Isleworth', 'Iver', 'Kenley', 'Kennington', 'Kensal Green', 'Kensal Rise', 'Kensington (Olympia)', 'Kent House', 'Kentish Town', 'Kentish Town West', 'Kenton', 'Kew Bridge', 'Kew Gardens', 'Kidbrooke', 'Kilburn', 'Kilburn High Road', 'Kilburn Park', 'King George V', "King Henry's Drive", "King's Cross", 'Kings Cross St. Pancras', 'Kingsbury', 'Kingston', 'Kingswood', 'Knightsbridge', 'Knockholt', 'Ladbroke Grove', 'Ladywell', 'Lambeth North', 'Lancaster Gate', 'Langdon Park', 'Langley', 'Latimer Road', 'Lea Bridge', 'Lebanon Road', 'Lee', 'Leicester Square', 'Lewisham', 'Leyton', 'Leyton Midland Road', 'Leytonstone', 'Leytonstone High Road', 'Limehouse', 'Liverpool Street', 'Lloyd Park', 'London Bridge', 'London City Airport', 'London Fields', 'Loughborough Junction', 'Loughton', 'Lower Sydenham', 'Maida Vale', 'Maidenhead', 'Malden Manor', 'Manor House', 'Manor Park', 'Mansion House', 'Marble Arch', 'Maryland', 'Marylebone', 'Maze Hill', 'Meridian Water', 'Merton Park', 'Mile End', 'Mill Hill Broadway', 'Mill Hill East', 'Mitcham', 'Mitcham Eastfields', 'Mitcham Junction', 'Monument', 'Moor Park', 'Moorgate', 'Morden', 'Morden Road', 'Morden South', 'Mornington Crescent', 'Mortlake', 'Motspur Park', 'Mottingham', 'Mudchute', 'Neasden', 'New Addington', 'New Barnet', 'New Beckenham', 'New Cross', 'New Cross Gate', 'New Eltham', 'New Malden', 'New Southgate', 'Newbury Park', 'Nine Elms', 'Norbiton', 'Norbury', 'North Acton', 'North Dulwich', 'North Ealing', 'North Greenwich', 'North Harrow', 'North Sheen', 'North Wembley', 'Northfields', 'Northolt', 'Northolt Park', 'Northumberland Park', 'Northwick Park', 'Northwood', 'Northwood Hills', 'Norwood Junction', 'Notting Hill Gate', 'Nunhead', 'Oakleigh Park', 'Oakwood', 'Ockendon', 'Old Street', 'Orpington', 'Osterley', 'Oval', 'Oxford Circus', 'Paddington', 'Palmers Green', 'Park Royal', 'Parsons Green', 'Peckham Rye', 'Penge East', 'Penge West', 'Perivale', 'Petts Wood', 'Phipps Bridge', 'Piccadilly Circus', 'Pimlico', 'Pinner', 'Plaistow', 'Plumstead', 'Ponders End', 'Pontoon Dock', 'Poplar', 'Preston Road', 'Prince Regent', 'Pudding Mill Lane', 'Purfleet', 'Purley', 'Purley Oaks', 'Putney', 'Putney Bridge', 'Queens Park', 'Queens Road Peckham', 'Queensbury', 'Queenstown Road', 'Queensway', 'Rainham', 'Ravensbourne', 'Ravenscourt Park', 'Rayners Lane', 'Raynes Park', 'Reading', 'Rectory Road', 'Redbridge', 'Reedham', 'Reeves Corner', 'Regents Park', 'Richmond', 'Rickmansworth', 'Riddlesdown', 'Roding Valley', 'Romford', 'Rotherhithe', 'Royal Albert', 'Royal Oak', 'Royal Victoria', 'Ruislip', 'Ruislip Gardens', 'Ruislip Manor', 'Russell Square', 'Sanderstead', 'Sandilands', 'Selhurst', 'Seven Kings', 'Seven Sisters', 'Shadwell', 'Shenfield', 'Shepherds Bush', 'Shepherds Bush Market', 'Shoreditch High Street', 'Shortlands', 'Sidcup', 'Silver Street', 'Slade Green', 'Sloane Square', 'Slough', 'Snaresbrook', 'South Acton', 'South Bermondsey', 'South Croydon', 'South Ealing', 'South Greenford', 'South Hampstead', 'South Harrow', 'South Kensington', 'South Kenton', 'South Merton', 'South Quay', 'South Ruislip', 'South Tottenham', 'South Wimbledon', 'South Woodford', 'Southall', 'Southbury', 'Southfields', 'Southgate', 'Southwark', 'St Helier', 'St James Street', 'St Johns', 'St Margarets', 'St Mary Cray', 'St Pancras', "St. James's Park", 'St. Johns Wood', 'St. Pauls', 'Stamford Brook', 'Stamford Hill', 'Stanmore', 'Star Lane', 'Stepney Green', 'Stockwell', 'Stoke Newington', 'Stonebridge Park', 'Stoneleigh', 'Stratford', 'Stratford High Street', 'Stratford International', 'Strawberry Hill', 'Streatham', 'Streatham Common', 'Streatham Hill', 'Sudbury and Harrow Road', 'Sudbury Hill', 'Sudbury Hill Harrow', 'Sudbury Town', 'Sundridge Park', 'Surbiton', 'Surrey Quays', 'Sutton', 'Sutton Common', 'Swiss Cottage', 'Sydenham', 'Sydenham Hill', 'Syon Lane', 'Tadworth', 'Taplow', 'Tattenham Corner', 'Teddington', 'Temple', 'Thames Ditton', 'Theobalds Grove', 'Therapia Lane', 'Theydon Bois', 'Thornton Heath', 'Tolworth', 'Tooting', 'Tooting Bec', 'Tooting Broadway', 'Tottenham Court Road', 'Tottenham Hale', 'Totteridge and Whetstone', 'Tower Gateway', 'Tower Hill', 'Tufnell Park', 'Tulse Hill', 'Turkey Street', 'Turnham Green', 'Turnpike Lane', 'Twickenham', 'Twyford', 'Upminster', 'Upminster Bridge', 'Upney', 'Upper Holloway', 'Upper Warlingham', 'Upton Park', 'Uxbridge', 'Vauxhall', 'Victoria', 'Waddon', 'Waddon Marsh', 'Wallington', 'Waltham Cross', 'Walthamstow Central', 'Walthamstow Queens Road', 'Wandle Park', 'Wandsworth Common', 'Wandsworth Road', 'Wandsworth Town', 'Wanstead', 'Wanstead Park', 'Wapping', 'Warren Street', 'Warwick Avenue', 'Waterloo', 'Waterloo East', 'Watford', 'Watford High Street', 'Watford Junction', 'Wellesley Road', 'Welling', 'Wembley Central', 'Wembley Park', 'Wembley Stadium', 'West Acton', 'West Brompton', 'West Croydon', 'West Drayton', 'West Dulwich', 'West Ealing', 'West Finchley', 'West Ham', 'West Hampstead', 'West Hampstead Thameslink', 'West Harrow', 'West India Quay', 'West Kensington', 'West Norwood', 'West Ruislip', 'West Silvertown', 'West Sutton', 'West Wickham', 'Westbourne Park', 'Westcombe Park', 'Westferry', 'Westminster', 'White City', 'White Hart Lane', 'Whitechapel', 'Whitton', 'Whyteleafe', 'Whyteleafe South', 'Willesden Green', 'Willesden Junction', 'Wimbledon', 'Wimbledon Chase', 'Wimbledon Park', 'Winchmore Hill', 'Wood Green', 'Wood Lane', 'Wood Street', 'Woodford', 'Woodgrange Park', 'Woodmansterne', 'Woodside', 'Woodside Park', 'Woolwich', 'Woolwich Arsenal', 'Woolwich Dockyard', 'Worcester Park']

if DISPLAY_TEST_TABLE: print("The following execution times are calculated from an average of", NOF_REPEATS, "repeats.\n")
testMinimumDistance(plotMapper, citiesToTest, NOF_REPEATS, NOF_TESTS, DISPLAY_TEST_TABLE)
if DISPLAY_TEST_TABLE: print("\n")
testMinimumStops(plotMapper, citiesToTest, NOF_REPEATS, NOF_TESTS, DISPLAY_TEST_TABLE)
if DISPLAY_TEST_TABLE: print("\n")
testNewRailwayLine(plotMapper, citiesToTest, NOF_REPEATS, NOF_TESTS, DISPLAY_TEST_TABLE)

The following execution times are calculated from an average of 3 repeats.

No. of stations - traversed    Average time taken             Minimum distance              
639                            0.012782887999999973           24.722398099404355            
270                            0.005710010333333256           9.630877611294707             
515                            0.011121898666666574           15.119270263511837            
536                            0.009825354000000027           13.083786403686807            
354                            0.005792598999999991           17.155013372201523            


No. of stations - traversed    Average time taken             Minimum stops                 
129                            0.00026202399999997183         6                             
317                            0.000568089999999947           11                            
120                            0.00025261099999992115         6                      

The cell below exemplifies the test code I will invoke on your submission. Do NOT modify it. 

In [5]:
# DO NOT MODIFY THIS CELL

import timeit

testMapper = LondonRailwayMapper()

#
# testing the loadStationsAndLines() API 
#
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

#
# testing the minStops() and minStops() API on a sample of from/to station pairs  
#
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")  
    
#
# testing the newRailwayLine() API on a small list of stations  
#
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

#
# testing the newRailwayLine() API on a big list of stations  
#
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


Execution time to load: 0.006

Execution time minStops: 0.0
Execution time minDistance: 0.0
From Baker Street to North Wembley in 6 stops and 6 miles

Execution time minStops: 0.001
Execution time minDistance: 0.001
From Epping to Belsize Park in 17 stops and 17 miles

Execution time minStops: 0.001
Execution time minDistance: 0.001
From Canonbury to Balham in 10 stops and 10 miles

Execution time minStops: 0.0
Execution time minDistance: 0.001
From Vauxhall to Leytonstone in 6 stops and 6 miles


Station list ['Queens Park', 'Chigwell', 'Moorgate', 'Swiss Cottage', 'Liverpool Street', 'Highgate']
New station line ['Chigwell', 'Liverpool Street', 'Moorgate', 'Swiss Cottage', 'Queens Park', 'Highgate']
Total track length from Chigwell to Highgate : 18.59580748923541 miles
Execution time newLine: 0.0


Station list ['Abbey Road', 'Barbican', 'Bethnal Green', 'Cambridge Heath', 'Covent Garden', 'Dollis Hill', 'East Finchley', 'Finchley Road and Frognal', 'Great Portland Street', 'Hackney